<a href="https://colab.research.google.com/github/Abhishek3253/Pyspark/blob/main/GroupBy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!apt-get install openjdk-8-jdk-headless -qq > /dev/null


!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz



!tar xf spark-3.2.1-bin-hadoop3.2.tgz

!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

import findspark
findspark.init()

findspark.find()

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Mounted at /content/drive


In [5]:
df = spark.read.csv("/content/assignment.csv",header=True ,inferSchema=True)

In [9]:
df.show(5)

+------+---------------+----------+------+--------+
|userid|        emailid|      date|amount|discount|
+------+---------------+----------+------+--------+
|     1|test1@gmail.com|01-01-2019|   100|     1.0|
|     2|test2@gmail.com|02-01-2019|   100|     1.0|
|     3|test3@gmail.com|03-01-2019|   101|     2.0|
|     4|test4@gmail.com|04-01-2019|   102|    10.0|
|     5|test5@gmail.com|05-01-2019| 102.5|     2.0|
+------+---------------+----------+------+--------+
only showing top 5 rows



In [11]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [41]:
from datetime import datetime
def func_year(text):
    for fmt in ('%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y','%d-%m-%Y','%dd-%mm-%yyyy'):
        try:
            date_object = datetime.strptime(text, fmt)
            converted_date_object = date_object.strftime('%Y')
            return converted_date_object

        except ValueError:
            pass
    return 'ERROR_DATE'

In [42]:
from datetime import datetime
def func_month(text):
    for fmt in ('%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y','%d-%m-%Y','%dd-%mm-%yyyy'):
        try:
            date_object = datetime.strptime(text, fmt)
            converted_date_object = date_object.strftime('%Y')
            return converted_date_object

        except ValueError:
            pass
    return 'ERROR_DATE'


In [46]:
year_df=udf(lambda x:func_year(x))
month_df=udf(lambda x:func_month(x))

In [47]:
df=df.withColumn("year",year_df("date"))
df=df.withColumn("month",month_df("date"))

In [48]:
df.show(truncate=False)

+------+----------------+----------+------+-----------+----+-----+--------+
|userid|emailid         |date      |amount|discount   |year|month|activity|
+------+----------------+----------+------+-----------+----+-----+--------+
|1     |test1@gmail.com |01-01-2019|100   |1.0        |2019|2019 |1       |
|2     |test2@gmail.com |02-01-2019|100   |1.0        |2019|2019 |1       |
|3     |test3@gmail.com |03-01-2019|101   |2.0        |2019|2019 |1       |
|4     |test4@gmail.com |04-01-2019|102   |10.0       |2019|2019 |1       |
|5     |test5@gmail.com |05-01-2019|102.5 |2.0        |2019|2019 |1       |
|6     |test6@gmail.com |06-01-2019|103.2 |1.0        |2019|2019 |1       |
|7     |test7@gmail.com |07-01-2019|103.9 |3.0        |2019|2019 |1       |
|8     |test8@gmail.com |08-01-2019|104.6 |5.0        |2019|2019 |1       |
|9     |test9@gmail.com |09-01-2019|105.3 |4.571428571|2019|2019 |1       |
|10    |test10@gmail.com|10-01-2019|106   |4.892857143|2019|2019 |1       |
|11    |test

In [49]:
df = df.withColumn('activity',lit(1))

In [50]:
df.show()

+------+----------------+----------+------+-----------+----+-----+--------+
|userid|         emailid|      date|amount|   discount|year|month|activity|
+------+----------------+----------+------+-----------+----+-----+--------+
|     1| test1@gmail.com|01-01-2019|   100|        1.0|2019| 2019|       1|
|     2| test2@gmail.com|02-01-2019|   100|        1.0|2019| 2019|       1|
|     3| test3@gmail.com|03-01-2019|   101|        2.0|2019| 2019|       1|
|     4| test4@gmail.com|04-01-2019|   102|       10.0|2019| 2019|       1|
|     5| test5@gmail.com|05-01-2019| 102.5|        2.0|2019| 2019|       1|
|     6| test6@gmail.com|06-01-2019| 103.2|        1.0|2019| 2019|       1|
|     7| test7@gmail.com|07-01-2019| 103.9|        3.0|2019| 2019|       1|
|     8| test8@gmail.com|08-01-2019| 104.6|        5.0|2019| 2019|       1|
|     9| test9@gmail.com|09-01-2019| 105.3|4.571428571|2019| 2019|       1|
|    10|test10@gmail.com|10-01-2019|   106|4.892857143|2019| 2019|       1|
|    11|test

In [51]:
#Filtering by GroupBy function
d_df = df.groupBy('year').agg(sum('amount').alias('sum'),min('amount'),count('activity'))

In [52]:
d_df.show()

+----------+------------------+-----------+---------------+
|      year|               sum|min(amount)|count(activity)|
+----------+------------------+-----------+---------------+
|      2019|           82654.9|        100|            366|
|      2020|176759.69999999998|      355.2|            366|
|      2021|          113263.8|      611.4|            169|
|ERROR_DATE|             258.6|      258.6|              1|
+----------+------------------+-----------+---------------+



# New Section